<a href="https://colab.research.google.com/github/PatSoto-BioPhys/PatSoto-BioPhys/blob/main/build2DdensitiesLower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install MDAnalysis

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import MDAnalysis as mda
import sys
import datetime
import math
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
!ls -ltr "/content/drive/My Drive"
#!unzip "/content/drive/My Drive/SM.zip" -d "/content"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 58122
-rw------- 1 root root      187 Aug  4  2015 'Log to keep a sense of accountability and help us all stay engaged, connected and informed.gdoc'
-rw------- 1 root root    95617 Sep  9  2015 'Project Progress & Goals.docx'
-rw------- 1 root root      187 May 24  2016 '3 29 16 - Prion term definitions.gdoc'
drwx------ 2 root root     4096 Apr 18  2018 'Frances Prion Videos'
-rw------- 1 root root      187 Jun  5  2018 'Implicit Solvent links.gdoc'
drwx------ 2 root root     4096 Sep  5  2018  Acetyltranferases
-rw------- 1 root root      187 Sep  5  2018 'Optimize gromacs in GPU.gdoc'
-rw------- 1 root root     7872 Apr  9  2019 'Asphericity_&_Radius_of_Gyration_4_4_2019.py'
-rw------- 1 root root      187 Apr 16  2019  Videos.gdoc
drwx------ 2 root root     4096 Jan 31  2020  PrepConferences
drwx------ 2 root root     4096 Feb 13  2020 'Hannah Brockm

In [ ]:
#uploaded = files.upload()
#uploadedTraj = files.upload()

Saving referencePrPC_SMpatch.gro to referencePrPC_SMpatch.gro


/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/usr/local/lib/python3.10/dist-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [ ]:
my_protein="Doppel"
my_patch = "PC"
u=mda.Universe('/content/drive/My Drive/PrPC_Doppel_membranes/reference'+my_protein+'_'+my_patch+'patch.gro',
               '/content/drive/My Drive/PrPC_Doppel_membranes/traj'+my_protein+'_'+my_patch+'patch.xtc')
#my_gro = pd.read_json(io.BytesIO(uploaded['referencePrPC_SMpatch.gro']), lines=True)

# USAGE *py [gro file] [traj file] [system header, include top/bot]

#u = MDAnalysis.Universe(sys.argv[1],sys.argv[2])
print(len(u.trajectory))

#

ppl = u.select_atoms('((resname POPC or resname POSM) and (name PO4 or name NC3)) or (resname POPG and (name PO4 or name GL0))')
#print(pc)

#

#sm = u.select_atoms('resname POSM and name PO4')

#

cl = u.select_atoms('name ROH')

#print(cl)



ct = datetime.datetime.now()
print("current time:*", ct)

for a in range(1,6):
  # collect timeseries
  ppl_ts = []
  #psm = []
  pcl = []
  #pma = []
  #rna = []
  x_box = []
  y_box = []

  for j, ts in enumerate(u.trajectory):
      #print("j= ", j)
      #print("ts =", ts)
      #print(f"x = {ts.dimensions[0]}, y = {ts.dimensions[1]}")

      x_box.append(ts.dimensions[0])
      y_box.append(ts.dimensions[1])

  max_x_box = numpy.max(x_box)
  max_y_box = numpy.max(y_box)

  print(f"average x: {numpy.mean(x_box)}")
  print(f"max x: {numpy.max(x_box)}\n")
  print(f"average y: {numpy.mean(y_box)}")
  print(f"max y: {numpy.max(y_box)}")

  count_leaflet_membership = []

  number_of_frames_analyzed = 0



  start = a * 3600
  end = start + 3600

  for ts in u.trajectory[start:end]:
    ppl_cog_z = ppl.center_of_geometry()[2]

    chol_cog_z = cl.center_of_geometry()[2]
    count = 0
    # lipids
    for ix in ppl:
        d = ix.position
        if d[0] > max_x_box:
              while True:
                  d[0] -= max_x_box
                  if d[0] <= max_x_box:
                      break

          #if y too big
        if d[1] > max_y_box:
              while True:
                  d[1] -= max_y_box
                  if d[1] <= max_y_box:
                      break

          #if x negative
        if d[0] < 0:
              while True:
                  d[0] += max_x_box
                  if d[0] >= 0:
                      break

          #if y negative
        if d[1] < 0:
              while True:
                  d[1] += max_y_box
                  if d[1] >= 0:
                      break
        if (d[2] < ppl_cog_z ):  #z coordinate of PC > center of geometry of membrane
          ppl_ts.append((d[0], d[1], d[2]))

       for ix in cl:
       d = ix.position
        #if x too big
       if d[0] > max_x_box:
            while True:
                d[0] -= max_x_box
                if d[0] <= max_x_box:
                    break

        #if y too big
       if d[1] > max_y_box:
            while True:
                d[1] -= max_y_box
                if d[1] <= max_y_box:
                    break

        #if x negative
       if d[0] < 0:
            while True:
                d[0] += max_x_box
                if d[0] >= 0:
                    break

        #if y negative
       if d[1] < 0:
            while True:
                d[1] += max_y_box
                if d[1] >= 0:
                    break
       if (d[2] < chol_cog_z ):  #z coordinate of ROH > center of geometry of membrane
       	pcl.append((d[0], d[1], d[2]))
        count += 1
    #
    #
    count_leaflet_membership.append(count)
    number_of_frames_analyzed += 1

  print('count_leaflet_membership ',count_leaflet_membership)
  print('number_of_frames_analyzed ', number_of_frames_analyzed)
  # for ix in ma:
  #     d = ix.position
  #     pma.append((d[0], d[1], d[2]))

  ppl_ts = np.array(ppl_ts)
  #psm = np.array(psm)
  pcl = np.array(pcl)
  #pma = np.array(pma)
  #rna = np.array(rna)

  ct = datetime.datetime.now()
  print("current time:*", ct)

  # SAVE DATA
  my_file='/content/drive/My Drive/PrPC_Doppel_membranes/'+my_protein+my_patch+'_ppl_NC3_lowerT'+ str(a)
  #np.savetxt('dspc_'+sys.argv[3]+'.dat', pcm, delimiter='\t', fmt='%2.4f', newline='\n')
  #np.savetxt('pope_'+sys.argv[3]+'.dat', pem, delimiter='\t', fmt='%2.4f', newline='\n')
  #np.savetxt('chol_'+sys.argv[3]+'.dat', pcl, delimiter='\t', fmt='%2.4f', newline='\n')
  np.savetxt(my_file+'.dat', ppl_ts, delimiter='\t', fmt='%2.4f', newline='\n')
  #np.savetxt('posm_upperT5.dat', psm, delimiter='\t', fmt='%2.4f', newline='\n')
  my_file_chol='/content/drive/My Drive/PrPC_Doppel_membranes/'+my_protein+my_patch+'_chol_lowerT'+ str(a)
  np.savetxt(my_file_chol+'.dat', pcl, delimiter='\t', fmt='%2.4f', newline='\n')
  #
  Nbins = 50
  pcm = ppl_ts
  plt.figure()
  ax = plt.subplot(111)
  #h,xx,yy = ax.hist2d(pcm[:,0],pcm[:,1], bins=Nbins, cmap='Greens', density='True', alpha=0.75, range=[[3,146],[3,146]])
  h,xx,yy = np.histogram2d(pcm[:,0],pcm[:,1], bins=Nbins,  range=[[3,146],[3,146]])
  h = h.T
  print("min popc histo= ",np.amin(h))
  plt.imshow(h, interpolation='spline36',cmap='viridis',alpha=0.9, origin='lower',extent=[xx[0],xx[-1],yy[0],yy[-1]])
  #v = ax.hist2d(p_residuo[:,0],p_residuo[:,1], bins=Nbins, cmap='Oranges', density='True', alpha=0.5, range=[[3,146],[3,146]])
  #sns.kdeplot(data=p_residuo, x=p_residuo[:,0], y=p_residuo[:,1], cmap='Greys', color='black', levels=1, ax=ax)
  #sns.kdeplot(data=p_residuo, x=p_residuo[:,0], y=p_residuo[:,1], color='black', levels=3, ax=ax)
  #sns.kdeplot(data=p_gpi, x=p_gpi[:,0], y=p_gpi[:,1], color='white', levels=3, ax=ax)
  #sns.kdeplot(data=p_residuo2, x=p_residuo2[:,0], y=p_residuo2[:,1], color='red', levels=3, ax=ax)
  plt.xlabel('x')
  plt.ylabel('y')
  #plt.tight_layout()
  plt.colorbar()

  ax.figure.savefig(my_file+'.png')
  plt.draw()
  plt.clf()
  plt.cla()
  plt.close()
  #
  h,xx,yy = np.histogram2d(pcl[:,0],pcl[:,1], bins=Nbins,  range=[[3,146],[3,146]])
  h = h.T
  #print("min popc histo= ",np.amin(h))
  plt.imshow(h, interpolation='spline36',cmap='viridis',alpha=0.9, origin='lower',extent=[xx[0],xx[-1],yy[0],yy[-1]])
  #v = ax.hist2d(p_residuo[:,0],p_residuo[:,1], bins=Nbins, cmap='Oranges', density='True', alpha=0.5, range=[[3,146],[3,146]])
  #sns.kdeplot(data=p_residuo, x=p_residuo[:,0], y=p_residuo[:,1], cmap='Greys', color='black', levels=1, ax=ax)
  #sns.kdeplot(data=p_residuo, x=p_residuo[:,0], y=p_residuo[:,1], color='black', levels=3, ax=ax)
  #sns.kdeplot(data=p_gpi, x=p_gpi[:,0], y=p_gpi[:,1], color='white', levels=3, ax=ax)
  #sns.kdeplot(data=p_residuo2, x=p_residuo2[:,0], y=p_residuo2[:,1], color='red', levels=3, ax=ax)
  plt.xlabel('x')
  plt.ylabel('y')
  #plt.tight_layout()
  plt.colorbar()

  ax.figure.savefig(my_file_chol+'.png')
  plt.draw()
  plt.clf()
  plt.cla()
  plt.close()



18005
current time:* 2023-09-17 22:33:27.278807
average x: 148.84140014648438
max x: 150.98863220214844

average y: 148.84140014648438
max y: 150.98863220214844
count_leaflet_membership  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(ppl_cog_z)

85.5091344510202


In [ ]:
!ls -ltr


total 621012
-rw-r--r-- 1 root root 580266000 Jul 12 17:23 trajPrPC_SMpatch.xtc
-rw-r--r-- 1 root root    395770 Jul 12 17:26 referencePrPC_SMpatch.gro
drwxr-xr-x 1 root root      4096 Sep 14 13:23 sample_data
drwx------ 5 root root      4096 Sep 17 12:46 drive
-rw-r--r-- 1 root root  55238276 Sep 17 13:03 1A_ppl_NC3_upperT4.dat


In [ ]:
!ls -ltr "/content/drive/My Drive"

total 678824
-rw------- 1 root root       187 Aug  4  2015 'Log to keep a sense of accountability and help us all stay engaged, connected and informed.gdoc'
-rw------- 1 root root     95617 Sep  9  2015 'Project Progress & Goals.docx'
-rw------- 1 root root       187 May 24  2016 '3 29 16 - Prion term definitions.gdoc'
drwx------ 2 root root      4096 Apr 18  2018 'Frances Prion Videos'
-rw------- 1 root root       187 Jun  5  2018 'Implicit Solvent links.gdoc'
drwx------ 2 root root      4096 Sep  5  2018  Acetyltranferases
-rw------- 1 root root       187 Sep  5  2018 'Optimize gromacs in GPU.gdoc'
-rw------- 1 root root      7872 Apr  9  2019 'Asphericity_&_Radius_of_Gyration_4_4_2019.py'
-rw------- 1 root root       187 Apr 16  2019  Videos.gdoc
drwx------ 2 root root      4096 Jan 31  2020  PrepConferences
drwx------ 2 root root      4096 Feb 13  2020 'Hannah Brockman'
-rw------- 1 root root       187 Mar 19  2020 'San Diego Report.gdoc'
-rw------- 1 root root  58943099 Aug 11  20